In [7]:
import numpy as np
import matplotlib.pyplot as plt
# Reuse the curvature estimator function
from scipy.spatial import cKDTree
from sklearn.linear_model import LinearRegression

In [60]:
def compute_second_derivatives(k_points, energies, k_target, delta=1e-4):
    tree = cKDTree(k_points)
    alpha_est = np.zeros(3)

    for i in range(3):
        k_plus = k_target.copy()
        k_plus[i] += delta
        k_minus = k_target.copy()
        k_minus[i] -= delta

        _, idx_plus = tree.query(k_plus)
        _, idx_minus = tree.query(k_minus)
        _, idx_center = tree.query(k_target)

        E_plus = energies[idx_plus]
        E_minus = energies[idx_minus]
        E_center = energies[idx_center]

        print(E_center)

        alpha_est[i] = (E_plus - 2 * E_center + E_minus) / (delta ** 2)
        print(alpha_est[i])

    return alpha_est

# Classify the critical point
def classify_critical_point(alphas):
    signs = np.sign(alphas)
    pos = np.sum(signs > 0)
    neg = np.sum(signs < 0)
    if pos == 3:
        return "Minimum"
    elif neg == 3:
        return "Maximum"
    elif pos == 2:
        return "Saddle point II"
    elif pos == 1:
        return "Saddle point I"
    else:
        return "Unknown"

In [70]:
# Simulate 3D k-space grid
k_vals = np.linspace(-0.1, 0.1, 100)
kx, ky, kz = np.meshgrid(k_vals, k_vals, k_vals, indexing='ij')
k_points = np.vstack([kx.ravel(), ky.ravel(), kz.ravel()]).T

In [71]:
# Puntos de alta simetría
Gamma = np.array([0, 0, 0])
P = np.array([0.5, 0.5, 0.5])
H = np.array([1, 0, 0])
N = np.array([0.5, 0.5, 0])
sym_points = [Gamma, H, P, N]

# Funciones de energía
G_vectors = [
    np.array([0, 0, 0]),
    np.array([1, 0, 0]),
    np.array([1, 1, 0]),
]
def E_free_e(k, G=G_vectors[0]): 
    return np.linalg.norm(k + G)**2 

def E_tight_binding(k, A = 0):
    return A + sum([np.cos(np.pi*k[i]) for i in range(3)])


energies = [(np.array([E_free_e(k) for k in (k_points.copy() + sym_points[i])])) for i in range(4)]

In [72]:
def fit_paraboloid(k_points, energies, k_center, radius=0.05):
    """
    Fit a quadratic surface E(k) = a0 + a1*kx + a2*ky + a3*kz + a4*kx^2 + a5*ky^2 + a6*kz^2
    to estimate second derivatives.
    """
    # Shift all k-points relative to k_center
    k_shifted = k_points - k_center
    mask = np.linalg.norm(k_shifted, axis=1) < radius
    k_local = k_shifted[mask]
    E_local = energies[mask]
    
    # Build the design matrix for a paraboloid (ignoring cross terms for simplicity)
    A = np.column_stack([
        np.ones(len(k_local)),        # a0
        k_local[:, 0],                # a1 * kx
        k_local[:, 1],                # a2 * ky
        k_local[:, 2],                # a3 * kz
        k_local[:, 0]**2,             # a4 * kx^2
        k_local[:, 1]**2,             # a5 * ky^2
        k_local[:, 2]**2              # a6 * kz^2
    ])

    model = LinearRegression().fit(A, E_local)
    coeffs = model.coef_
    
    # Second derivatives: 2*a4, 2*a5, 2*a6
    alpha = 2 * np.array([coeffs[4], coeffs[5], coeffs[6]])
    return alpha

In [73]:
# Run estimation
for i in range(4):
    alpha_estimated = fit_paraboloid((k_points.copy() + sym_points[i]), energies[i], sym_points[i], 0.1)
    print("Estimated α values: ", alpha_estimated)
    print("Classification: ", classify_critical_point(alpha_estimated))

Estimated α values:  [2. 2. 2.]
Classification:  Minimum
Estimated α values:  [2. 2. 2.]
Classification:  Minimum
Estimated α values:  [2. 2. 2.]
Classification:  Minimum
Estimated α values:  [2. 2. 2.]
Classification:  Minimum
